## Crecimiento económico de largo plazo.
### Hoy tilizaremos Python, pandas y Matplotlib para descargar, organizar y visualizar datos históricos sobre el crecimiento económico.

Además de aprender a utilizar estas herramientas de forma más general, las usaremos para describir hechos sobre experiencias de crecimiento económico en muchos países a lo largo de varios siglos.

Estos "datos de crecimiento" son interesantes por varias razones.

Explicar los hechos del crecimiento es uno de los principales objetivos tanto de la "economía del desarrollo" como de la "historia económica".

Además, los datos sobre el crecimiento son importantes para los historiadores a la hora de estudiar las fuerzas y dinámicas geopolíticas.

Estos gráficos mostrarán cómo la "Revolución Industrial" comenzó en Gran Bretaña a finales del siglo XVIII y cómo fue migrando de un país a otro.

En pocas palabras, esta conferencia registra las trayectorias de crecimiento de varios países durante largos periodos de tiempo.

Mientras que algunos países han experimentado un rápido crecimiento a largo plazo que ha durado cien años, otros no.

Dado que las poblaciones difieren de un país a otro y varían dentro de un mismo país a lo largo del tiempo, será interesante describir tanto el PIB total como el PIB per cápita a medida que evolucionan dentro de un mismo país.

Primero vamos a importar los paquetes necesarios para explorar lo que dicen los datos sobre el crecimiento a largo plazo

In [64]:
import pandas as pd
import matplotlib.pyplot as plt
import matplotlib.cm as cm
import numpy as np
import os
from collections import namedtuple

In [65]:
# Obtener el directorio actual (notebooks)
current_dir = os.getcwd()

# Navegar un nivel hacia arriba para llegar al directorio base del proyecto
base_dir = os.path.dirname(current_dir)

# Construir la ruta al archivo CSV en la carpeta df/external
xlsx_path = os.path.join(base_dir, 'data', 'external', 'mpd2020.xlsx')


# Leer el archivo CSV usando pandas
df = pd.read_excel(xlsx_path,sheet_name='Full data')

# Mostrar las primeras filas del dfFrame

df.head()

,countrycode,country,year,gdppc,pop
0,AFG,Afghanistan,1820,NaN,3280.0
1,AFG,Afghanistan,1870,NaN,4207.0
2,AFG,Afghanistan,1913,NaN,5730.0
3,AFG,Afghanistan,1950,1156.0,8150.0
4,AFG,Afghanistan,1951,1170.0,8284.0


In [66]:
#Forma del dfset
df.shape

(21682, 5)

In [67]:
# Información sobre los tipos de datos
df.dtypes

countrycode     object
country         object
year             int64
gdppc          float64
pop            float64
dtype: object

In [68]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 21682 entries, 0 to 21681
Data columns (total 5 columns):
 #   Column       Non-Null Count  Dtype  
---  ------       --------------  -----  
 0   countrycode  21682 non-null  object 
 1   country      21682 non-null  object 
 2   year         21682 non-null  int64  
 3   gdppc        19706 non-null  float64
 4   pop          17199 non-null  float64
dtypes: float64(2), int64(1), object(2)
memory usage: 847.1+ KB


In [69]:
df.columns

Index(['countrycode', 'country', 'year', 'gdppc', 'pop'], dtype='object')

### Podemos realizar un analisis informativo y descriptivo del dataset. 

In [70]:
df.describe()

,year,gdppc,pop
count,21682.000000,19706.000000,1.719900e+04
mean,1880.124297,6693.285574,2.980356e+04
std,187.856176,10111.643395,9.842090e+04
min,1.000000,0.000000,2.000000e+00
25%,1861.000000,1553.000000,2.235000e+03
50%,1954.000000,2794.000000,5.830000e+03
75%,1986.000000,7099.541625,1.889884e+04
max,2018.000000,156299.000000,1.385439e+06


In [71]:
#Podemos observar que tenemos valores nulos en el PBI per capita y la población.
df.isna().sum()

countrycode       0
country           0
year              0
gdppc          1976
pop            4483
dtype: int64

In [72]:
df.duplicated().sum()

0

Podemos ver que este conjunto de datos contiene el PIB per cápita (gdppc) y la población (pop) de muchos países y años.

Veamos cuántos y qué países están disponibles en este conjunto de datos

In [73]:
#Podemos ver que contamos con 169 paises usando el metodo unique.
countries = df['country'].unique()
len(countries) #Con len lo transformamos a una lista

169

Ahora podemos explorar algunos de los 169 países disponibles.

Hagamos un bucle sobre cada país para saber qué años están disponibles para cada país

In [74]:
df.tail()

,countrycode,country,year,gdppc,pop
21677,ZWE,Zimbabwe,2014,1594.0000,13313.99205
21678,ZWE,Zimbabwe,2015,1560.0000,13479.13812
21679,ZWE,Zimbabwe,2016,1534.0000,13664.79457
21680,ZWE,Zimbabwe,2017,1582.3662,13870.26413
21681,ZWE,Zimbabwe,2018,1611.4052,14096.61179


In [75]:
country_years = []
for country in countries:
    cy_df = df[df.country == country]['year']
    ymin, ymax = cy_df.min(), cy_df.max()
    country_years.append((country, ymin, ymax))
country_years = pd.DataFrame(country_years,
                    columns=['country', 'min_year', 'max_year']).set_index('country')
country_years.head()

,min_year,max_year
country,,
Afghanistan,1820,2018
Angola,1950,2018
Albania,1,2018
United Arab Emirates,1950,2018
Argentina,1800,2018


Ahora vamos a transformar los datos originales en algunas variables convenientes para permitir un acceso más rápido a los datos de las series temporales de los países.

Podemos crear una correspondencia útil entre los códigos de los países y sus nombres en este conjunto de datos

In [76]:
# Crea un DataFrame 'code_to_name' a partir de 'df' seleccionando solo las columnas 'countrycode' y 'country'.
# Usa drop_duplicates() para eliminar filas duplicadas basadas en estas dos columnas, asegurándose de que cada código de país esté asociado con un solo nombre de país.
# reset_index(drop=True) reinicia el índice del DataFrame resultante para que sea numérico y consecutivo, eliminando el índice anterior.
# set_index(['countrycode']) establece 'countrycode' como el nuevo índice del DataFrame, facilitando la búsqueda por código de país.
code_to_name = df[
    ['countrycode', 'country']].drop_duplicates().reset_index(drop=True).set_index(['countrycode'])
code_to_name

,country
countrycode,
AFG,Afghanistan
AGO,Angola
ALB,Albania
ARE,United Arab Emirates
ARG,Argentina
...,...
YEM,Yemen
YUG,Former Yugoslavia
ZAF,South Africa


Ahora podemos centrarnos en el PIB per cápita (gdppc) y generar un amplio formato de datos

In [77]:
# Crea un DataFrame 'gdp_pc' desde 'df'.
# set_index(['countrycode', 'year']) establece un índice múltiple basado en 'countrycode' y 'year', lo que permite una estructura de datos jerárquica que es útil para series de tiempo y otros análisis multidimensionales.
# Se selecciona la columna 'gdppc' (supuestamente Producto Interno Bruto per cápita).
# unstack('countrycode') cambia la estructura del DataFrame al convertir el nivel del índice 'countrycode' en columnas, lo que produce un DataFrame donde cada columna representa el PIB per cápita de un país y cada fila representa un año diferente.
gdp_pc = df.set_index(['countrycode', 'year'])['gdppc']
gdp_pc = gdp_pc.unstack('countrycode')

gdp_pc.tail()

countrycode,AFG,AGO,ALB,ARE,ARG,ARM,AUS,AUT,AZE,BDI,...,URY,USA,UZB,VEN,VNM,YEM,YUG,ZAF,ZMB,ZWE
year,,,,,,,,,,,,,,,,,,,,,
2014,2022.0000,8673.0000,9808.0000,72601.0000,19183.0000,9735.0000,47867.0000,41338.0000,17439.0000,748.0000,...,19160.0000,51664.0000,9085.0000,20317.0000,5455.0000,4054.0000,14627.0000,12242.0000,3478.0000,1594.0000
2015,1928.0000,8689.0000,10032.0000,74746.0000,19502.0000,10042.0000,48357.0000,41294.0000,17460.0000,694.0000,...,19244.0000,52591.0000,9720.0000,18802.0000,5763.0000,2844.0000,14971.0000,12246.0000,3478.0000,1560.0000
2016,1929.0000,8453.0000,10342.0000,75876.0000,18875.0000,10080.0000,48845.0000,41445.0000,16645.0000,665.0000,...,19468.0000,53015.0000,10381.0000,15219.0000,6062.0000,2506.0000,15416.0000,12139.0000,3479.0000,1534.0000
2017,2014.7453,8146.4354,10702.1201,76643.4984,19200.9061,10859.3783,49265.6135,42177.3706,16522.3072,671.3169,...,19918.1361,54007.7698,10743.8666,12879.1350,6422.0865,2321.9239,15960.8432,12189.3579,3497.5818,1582.3662
2018,1934.5550,7771.4418,11104.1665,76397.8181,18556.3831,11454.4251,49830.7993,42988.0709,16628.0553,651.3589,...,20185.8360,55334.7394,11220.3702,10709.9506,6814.1423,2284.8899,16558.3123,12165.7948,3534.0337,1611.4052


Creamos una variable color_mapping para almacenar un mapa entre códigos de país y colores por coherencia

In [78]:
# 'country_names' obtiene una serie de pandas que contiene los códigos de los países de la columna 'countrycode'.
country_names = df['countrycode']

# Genera un mapa de colores usando 'tab20', que es un mapa de colores categórico adecuado para datos cualitativos.
# 'np.linspace(0, 0.95, len(country_names))' genera una secuencia de valores entre 0 y 0.95,
# con tantos valores como número de países, asegurando que cada país tenga un color único.
colors = cm.tab20(np.linspace(0, 0.95, len(country_names)))

# Crea un diccionario que asocia cada código de país con un color correspondiente.
# Esto se logra mediante un bucle que recorre simultáneamente 'country_names' y 'colors'.
color_mapping = {country: color for country, color in zip(country_names, colors)}

In [ ]:
%store gdp_pc  color_mapping code_to_name


Stored 'gdp_pc' (DataFrame)
Stored 'color_mapping' (dict)
Stored 'code_to_name' (DataFrame)
